In [ ]:
# Load S&P 500 tickers
import pandas as pd

sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
sp500_table = pd.read_html(sp500_url)[0]
tickers = sp500_table['Symbol'].tolist()

# Download 2 years of close price data
import yfinance as yf
data = yf.download(tickers, period='2y', interval='1d')['Close']

# Classify each stock into trend types (reusing earlier function)
def classify_stock(prices):
    returns = prices.pct_change().dropna()
    volatility = returns.std()
    total_return = (prices.iloc[-1] - prices.iloc[0]) / prices.iloc[0]
    
    if abs(total_return) < 0.05 and volatility < 0.02:
        return 'stagnant'
    elif abs(total_return) > 0.2 and volatility < 0.03:
        return 'steady'
    else:
        return 'variable'

stock_classification = {}
for ticker in data.columns:
    trend_type = classify_stock(data[ticker])
    stock_classification[ticker] = trend_type

classification_df = pd.DataFrame.from_dict(stock_classification, orient='index', columns=['Trend Type'])
print(classification_df)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  503 of 503 completed

2 Failed downloads:
['BF.B']: YFPricesMissingError('possibly delisted; no price data found  (period=2y)')
['BRK.B']: YFPricesMissingError('possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")')
C:\Users\kober\AppData\Local\Temp\ipykernel_15892\2050643408.py:14: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  returns = prices.pct_change().dropna()


     Trend Type
A        steady
AAPL     steady
ABBV   variable
ABNB   variable
ABT      steady
...         ...
XYL    variable
YUM    variable
ZBH      steady
ZBRA     steady
ZTS    variable

[503 rows x 1 columns]


## Picking 5 most popular stocks from each trend type

In [2]:
import yfinance as yf

def get_popularity_info(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        return {
            'Ticker': ticker,
            'Market Cap': info.get('marketCap', 0),
            'Avg Volume': info.get('averageVolume', 0)
        }
    except:
        return {
            'Ticker': ticker,
            'Market Cap': 0,
            'Avg Volume': 0
        }


# Filter steady stocks, get their popularity info
steady_stocks = classification_df[classification_df['Trend Type'] == 'steady'].index.tolist()
popularity_data = [get_popularity_info(ticker) for ticker in steady_stocks]
popularity_df = pd.DataFrame(popularity_data)

top_steady = popularity_df.sort_values(by='Market Cap', ascending=False).head(5)['Ticker'].tolist()
print("Top 5 popular steady stocks:", top_steady)


# Filter steady stocks, get their popularity info
variable_stocks = classification_df[classification_df['Trend Type'] == 'variable'].index.tolist()
popularity_data = [get_popularity_info(ticker) for ticker in variable_stocks]
popularity_df = pd.DataFrame(popularity_data)

top_variable = popularity_df.sort_values(by='Market Cap', ascending=False).head(5)['Ticker'].tolist()
print("Top 5 popular variable stocks:", top_variable)


# Filter steady stocks, get their popularity info
stagnant_stocks = classification_df[classification_df['Trend Type'] == 'stagnant'].index.tolist()
popularity_data = [get_popularity_info(ticker) for ticker in stagnant_stocks]
popularity_df = pd.DataFrame(popularity_data)

top_stagnant = popularity_df.sort_values(by='Market Cap', ascending=False).head(5)['Ticker'].tolist()
print("Top 5 popular stagnant stocks:", top_stagnant)

Top 5 popular steady stocks: ['AAPL', 'MSFT', 'GOOG', 'GOOGL', 'AMZN']


401 Client Error: Unauthorized for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/XEL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=XEL&crumb=Edge%3A+Too+Many+Requests


Top 5 popular variable stocks: ['ABBV', 'ADBE', 'ADI', 'ABNB', 'ZTS']
Top 5 popular stagnant stocks: ['XOM', 'JNJ', 'ACN', 'CMCSA', 'LMT']


## Picking 5 random stocks from each trend type

In [3]:
# Set number of stocks you want from each trend type
n_variable = 5
n_stagnant = 5
n_steady = 5

# Sample stocks
sampled_variable = classification_df[classification_df['Trend Type'] == 'variable'].sample(n=n_variable, random_state=42)
sampled_stagnant = classification_df[classification_df['Trend Type'] == 'stagnant'].sample(n=n_stagnant, random_state=42)
sampled_steady = classification_df[classification_df['Trend Type'] == 'steady'].sample(n=n_steady, random_state=42)

# Combine
final_selection = pd.concat([sampled_variable, sampled_stagnant, sampled_steady])

# List of selected tickers
selected_tickers = final_selection.index.tolist()

print("Selected tickers:")
print(selected_tickers)

Selected tickers:
['NXPI', 'OMC', 'UHS', 'FE', 'MPC', 'XOM', 'EG', 'MDT', 'PFG', 'EXPD', 'MAR', 'IRM', 'FI', 'NEM', 'CME']
